In [58]:
import csv
import numpy
import os
import yaml
import time

In [69]:
from nupic.algorithms.sdr_classifier_factory import SDRClassifierFactory
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory
from nupic.encoders.random_distributed_scalar import RandomDistributedScalarEncoder

In [70]:
_NUM_RECORDS = 1000

_WORK_DIR = os.getcwd()
_INPUT_FILE_PATH = os.path.join(_WORK_DIR, "dataset", "eyeState14400.csv")
_PARAMS_PATH = os.path.join(_WORK_DIR, "params", "modelEyeState.yaml")

print _INPUT_FILE_PATH
print _PARAMS_PATH

E:\vo\kuliahS1\ta\eyeStateHTM\dataset\eyeState14400.csv
E:\vo\kuliahS1\ta\eyeStateHTM\params\modelEyeState.yaml


In [71]:
with open(_PARAMS_PATH, "r") as f:
    modelParams = yaml.safe_load(f)["modelParams"]
    enParams = modelParams["sensorParams"]["encoders"]
    spParams = modelParams["spParams"]
    tmParams = modelParams["tmParams"]

In [72]:
encoderAF3 = RandomDistributedScalarEncoder(enParams["sensAF3"]["resolution"])
encoderF7 = RandomDistributedScalarEncoder(enParams["sensF7"]["resolution"])
encoderF3 = RandomDistributedScalarEncoder(enParams["sensF3"]["resolution"])
encoderFC5 = RandomDistributedScalarEncoder(enParams["sensFC5"]["resolution"])
encoderT7 = RandomDistributedScalarEncoder(enParams["sensT7"]["resolution"])
encoderP7 = RandomDistributedScalarEncoder(enParams["sensP7"]["resolution"])
encoderO1 = RandomDistributedScalarEncoder(enParams["sensO1"]["resolution"])
encoderO2 = RandomDistributedScalarEncoder(enParams["sensO2"]["resolution"])
encoderP8 = RandomDistributedScalarEncoder(enParams["sensP8"]["resolution"])
encoderT8 = RandomDistributedScalarEncoder(enParams["sensT8"]["resolution"])
encoderFC6 = RandomDistributedScalarEncoder(enParams["sensFC6"]["resolution"])
encoderF4 = RandomDistributedScalarEncoder(enParams["sensF4"]["resolution"])
encoderF8 = RandomDistributedScalarEncoder(enParams["sensF8"]["resolution"])
encoderAF4 = RandomDistributedScalarEncoder(enParams["sensAF4"]["resolution"])
encoderEyeDetection = RandomDistributedScalarEncoder(enParams["sensEyeDetection"]["resolution"])

encodingWidth = (encoderAF3.getWidth() + encoderF7.getWidth() + encoderF3.getWidth()
                 + encoderFC5.getWidth() + encoderT7.getWidth() + encoderP7.getWidth()
                 + encoderO1.getWidth() + encoderO2.getWidth() + encoderP8.getWidth()
                 + encoderT8.getWidth() + encoderFC6.getWidth() + encoderF4.getWidth()
                 + encoderF8.getWidth() + encoderAF4.getWidth() + encoderEyeDetection.getWidth())

print encodingWidth
print encoderEyeDetection
print encoderAF3

6000
RandomDistributedScalarEncoder:
  minIndex:   500
  maxIndex:   500
  w:          21
  n:          400
  resolution: 0.88
  offset:     None
  numTries:   0
  name:       [0.88]
RandomDistributedScalarEncoder:
  minIndex:   500
  maxIndex:   500
  w:          21
  n:          400
  resolution: 0.88
  offset:     None
  numTries:   0
  name:       [0.88]


In [73]:
start = time.time()

sp = SpatialPooler(
  inputDimensions=(encodingWidth,),
  columnDimensions=(spParams["columnCount"],),
  potentialPct=spParams["potentialPct"],
  potentialRadius=encodingWidth,
  globalInhibition=spParams["globalInhibition"],
  localAreaDensity=spParams["localAreaDensity"],
  numActiveColumnsPerInhArea=spParams["numActiveColumnsPerInhArea"],
  synPermInactiveDec=spParams["synPermInactiveDec"],
  synPermActiveInc=spParams["synPermActiveInc"],
  synPermConnected=spParams["synPermConnected"],
  boostStrength=spParams["boostStrength"],
  seed=spParams["seed"],
  wrapAround=False
)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

00:00:48.50


In [74]:
tm = TemporalMemory(
  columnDimensions=(tmParams["columnCount"],),
  cellsPerColumn=tmParams["cellsPerColumn"],
  activationThreshold=tmParams["activationThreshold"],
  initialPermanence=tmParams["initialPerm"],
  connectedPermanence=spParams["synPermConnected"],
  minThreshold=tmParams["minThreshold"],
  maxNewSynapseCount=tmParams["newSynapseCount"],
  permanenceIncrement=tmParams["permanenceInc"],
  permanenceDecrement=tmParams["permanenceDec"],
  predictedSegmentDecrement=0.0,
  maxSegmentsPerCell=tmParams["maxSegmentsPerCell"],
  maxSynapsesPerSegment=tmParams["maxSynapsesPerSegment"],
  seed=tmParams["seed"]
)
print encodingWidth

6000


In [75]:
classifier = SDRClassifierFactory.create()
wrongDetect = 0

In [76]:
with open(_INPUT_FILE_PATH, "r") as fin:
  reader = csv.reader(fin)
  headers = reader.next()
  reader.next()
  reader.next()

  for count, record in enumerate(reader):

    if count >= _NUM_RECORDS: break
    
    sensAF3 = float(record[0])
    sensF7 = float(record[1])
    sensF3 = float(record[2])
    sensFC5 = float(record[3])
    sensT7 = float(record[4])
    sensP7 = float(record[5])
    sensO1 = float(record[6])
    sensO2 = float(record[7])
    sensP8 = float(record[8])
    sensT8 = float(record[9])
    sensFC6 = float(record[10])
    sensF4 = float(record[11])
    sensF8 = float(record[12])
    sensAF4 = float(record[13])
    sensEyeDetection = float(record[14])
    
    sensAF3_Bits = numpy.zeros(encoderAF3.getWidth())
    sensF7_Bits = numpy.zeros(encoderF7.getWidth()) 
    sensF3_Bits = numpy.zeros(encoderF3.getWidth()) 
    sensFC5_Bits = numpy.zeros(encoderFC5.getWidth())
    sensT7_Bits = numpy.zeros(encoderT7.getWidth()) 
    sensP7_Bits = numpy.zeros(encoderP7.getWidth()) 
    sensO1_Bits = numpy.zeros(encoderO1.getWidth()) 
    sensO2_Bits = numpy.zeros(encoderO2.getWidth()) 
    sensP8_Bits = numpy.zeros(encoderP8.getWidth()) 
    sensT8_Bits = numpy.zeros(encoderT8.getWidth()) 
    sensFC6_Bits = numpy.zeros(encoderFC6.getWidth())
    sensF4_Bits = numpy.zeros(encoderF4.getWidth()) 
    sensF8_Bits = numpy.zeros(encoderF8.getWidth()) 
    sensAF4_Bits = numpy.zeros(encoderAF4.getWidth())
    sensEyeDetection_Bits = numpy.zeros(encoderEyeDetection.getWidth())
    
    encoderAF3.encodeIntoArray(sensAF3, sensAF3_Bits)
    encoderF7.encodeIntoArray(sensF7, sensF7_Bits)
    encoderF3.encodeIntoArray(sensF3, sensF3_Bits)
    encoderFC5.encodeIntoArray(sensFC5, sensFC5_Bits)
    encoderT7.encodeIntoArray(sensT7, sensT7_Bits)
    encoderP7.encodeIntoArray(sensP7, sensP7_Bits)
    encoderO1.encodeIntoArray(sensO1, sensO1_Bits)
    encoderO2.encodeIntoArray(sensO2, sensO2_Bits)
    encoderP8.encodeIntoArray(sensP8, sensP8_Bits)
    encoderT8.encodeIntoArray(sensT8, sensT8_Bits)
    encoderFC6.encodeIntoArray(sensFC6, sensFC6_Bits)
    encoderF4.encodeIntoArray(sensF4, sensF4_Bits)
    encoderF8.encodeIntoArray(sensF8, sensF8_Bits)
    encoderAF4.encodeIntoArray(sensAF4, sensAF4_Bits)
    encoderEyeDetection.encodeIntoArray(sensEyeDetection, sensEyeDetection_Bits)

    encoding = numpy.concatenate(
        [sensAF3_Bits, sensF7_Bits, sensF3_Bits, sensFC5_Bits, sensT7_Bits,
         sensP7_Bits, sensO1_Bits, sensO2_Bits, sensP8_Bits, sensT8_Bits,
         sensFC6_Bits, sensF4_Bits, sensF8_Bits, sensAF4_Bits, sensEyeDetection_Bits]
    )
    
    activeColumns = numpy.zeros(spParams["columnCount"])
    
    # Execute Spatial Pooling algorithm over input space.
    sp.compute(encoding, True, activeColumns)
    activeColumnIndices = numpy.nonzero(activeColumns)[0]

    # Execute Temporal Memory algorithm over active mini-columns.
    tm.compute(activeColumnIndices, learn=True)

    activeCells = tm.getActiveCells()
    
    #print encoding
    
    # Get the bucket info for this input value for classification.
    bucketIdx = encoderEyeDetection.getBucketIndices(sensEyeDetection)[0]
    
    # Run classifier to translate active cells back to scalar value.
    classifierResult = classifier.compute(
      recordNum=count,
      patternNZ=activeCells,
      classification={
        "bucketIdx": bucketIdx,
        "actValue": sensEyeDetection
      },
      learn=True,
      infer=True
    )

    # Print the best prediction for 1 step out.
    oneStepConfidence, oneStep = sorted(
      zip(classifierResult[1], classifierResult["actualValues"]),
      reverse=True
    )[0]
    print("{:5}\t{:4}\t{:4}\t({:4.4}%)".format(count, sensEyeDetection, oneStep, oneStepConfidence * 100))
    #results.append([oneStep, oneStepConfidence * 100, None, None])
    if (sensEyeDetection == oneStep) : wrongDetect += 1
    #print classifierResult



    0	 0.0	 0.0	(100.0%)
    1	 0.0	 0.0	(0.1996%)
    2	 0.0	 0.0	(0.1996%)
    3	 0.0	 0.0	(0.1996%)
    4	 0.0	 0.0	(0.1996%)
    5	 0.0	 0.0	(0.1996%)
    6	 0.0	 0.0	(0.1996%)
    7	 0.0	 0.0	(0.1996%)
    8	 0.0	 0.0	(0.1996%)
    9	 0.0	 0.0	(0.1996%)
   10	 0.0	 0.0	(0.2061%)
   11	 0.0	 0.0	(0.1996%)
   12	 0.0	 0.0	(0.266%)
   13	 0.0	 0.0	(0.2747%)
   14	 0.0	 0.0	(0.2746%)
   15	 0.0	 0.0	(0.8632%)
   16	 0.0	 0.0	(0.7136%)
   17	 0.0	 0.0	(0.9193%)
   18	 0.0	 0.0	(0.836%)
   19	 0.0	 0.0	(0.8099%)
   20	 0.0	 0.0	(0.7142%)
   21	 0.0	 0.0	(0.6919%)
   22	 0.0	 0.0	(1.22%)
   23	 0.0	 0.0	(1.258%)
   24	 0.0	 0.0	(4.089%)
   25	 0.0	 0.0	(5.337%)
   26	 0.0	 0.0	(1.803%)
   27	 0.0	 0.0	(5.293%)
   28	 0.0	 0.0	(4.82%)
   29	 0.0	 0.0	(7.892%)
   30	 0.0	 0.0	(6.128%)
   31	 0.0	 0.0	(4.177%)
   32	 0.0	 0.0	(8.249%)
   33	 0.0	 0.0	(5.294%)
   34	 0.0	 0.0	(5.408%)
   35	 0.0	 0.0	(11.08%)
   36	 0.0	 0.0	(22.76%)
   37	 0.0	 0.0	(2.007%)
   38	 0.0	 0.0	(25.54%)
   39	 0

  327	 1.0	 0.0	(1.631%)
  328	 1.0	 1.0	(46.72%)
  329	 1.0	 1.0	(1.876%)
  330	 1.0	 1.0	(7.164%)
  331	 1.0	 0.0	(57.6%)
  332	 1.0	 0.0	(0.5562%)
  333	 1.0	 1.0	(0.4524%)
  334	 1.0	 1.0	(50.98%)
  335	 1.0	 1.0	(0.7595%)
  336	 1.0	 1.0	(9.55%)
  337	 1.0	 1.0	(95.09%)
  338	 1.0	 1.0	(5.082%)
  339	 1.0	 0.0	(50.62%)
  340	 1.0	 0.0	(1.23%)
  341	 1.0	 1.0	(5.923%)
  342	 1.0	 1.0	(72.84%)
  343	 1.0	 1.0	(0.6989%)
  344	 1.0	 1.0	(2.42%)
  345	 1.0	 1.0	(76.24%)
  346	 1.0	 1.0	(89.77%)
  347	 1.0	 1.0	(54.97%)
  348	 1.0	 1.0	(0.6955%)
  349	 1.0	 0.0	(20.16%)
  350	 1.0	 1.0	(55.82%)
  351	 1.0	 0.0	(1.916%)
  352	 1.0	 1.0	(38.99%)
  353	 1.0	 1.0	(0.7591%)
  354	 1.0	 1.0	(26.22%)
  355	 1.0	 0.0	(3.398%)
  356	 1.0	 1.0	(26.27%)
  357	 1.0	 1.0	(1.603%)
  358	 1.0	 1.0	(1.882%)
  359	 1.0	 0.0	(2.261%)
  360	 1.0	 0.0	(60.84%)
  361	 1.0	 0.0	(3.818%)
  362	 1.0	 0.0	(1.173%)
  363	 1.0	 0.0	(1.277%)
  364	 1.0	 0.0	(2.922%)
  365	 1.0	 0.0	(0.372%)
  366	 1.0	 1.0	(0.2742

  657	 1.0	 1.0	(96.63%)
  658	 1.0	 1.0	(99.88%)
  659	 1.0	 1.0	(98.89%)
  660	 1.0	 1.0	(98.71%)
  661	 1.0	 1.0	(98.95%)
  662	 1.0	 1.0	(99.76%)
  663	 1.0	 1.0	(99.69%)
  664	 1.0	 1.0	(99.48%)
  665	 1.0	 1.0	(99.96%)
  666	 1.0	 1.0	(99.72%)
  667	 1.0	 1.0	(95.18%)
  668	 1.0	 1.0	(95.52%)
  669	 1.0	 1.0	(84.0%)
  670	 1.0	 1.0	(99.54%)
  671	 1.0	 1.0	(99.41%)
  672	 1.0	 1.0	(99.81%)
  673	 1.0	 1.0	(99.91%)
  674	 1.0	 1.0	(99.74%)
  675	 1.0	 1.0	(84.93%)
  676	 1.0	 1.0	(97.27%)
  677	 1.0	 1.0	(99.77%)
  678	 1.0	 1.0	(99.43%)
  679	 1.0	 1.0	(99.67%)
  680	 1.0	 1.0	(98.82%)
  681	 1.0	 1.0	(97.35%)
  682	 1.0	 1.0	(99.9%)
  683	 1.0	 1.0	(98.69%)
  684	 1.0	 1.0	(99.89%)
  685	 1.0	 1.0	(85.61%)
  686	 1.0	 1.0	(99.01%)
  687	 1.0	 1.0	(84.85%)
  688	 1.0	 1.0	(99.37%)
  689	 1.0	 1.0	(85.7%)
  690	 1.0	 1.0	(99.93%)
  691	 1.0	 1.0	(99.9%)
  692	 1.0	 1.0	(99.86%)
  693	 1.0	 1.0	(99.09%)
  694	 1.0	 1.0	(99.78%)
  695	 1.0	 1.0	(99.29%)
  696	 1.0	 1.0	(91.58%)
  69

  983	 0.0	 0.0	(0.5182%)
  984	 0.0	 0.0	(1.144%)
  985	 0.0	 0.0	(65.19%)
  986	 0.0	 0.0	(99.94%)
  987	 0.0	 0.0	(90.37%)
  988	 0.0	 0.0	(14.02%)
  989	 0.0	 1.0	(2.719%)
  990	 0.0	 0.0	(3.049%)
  991	 0.0	 0.0	(0.8771%)
  992	 0.0	 1.0	(1.298%)
  993	 0.0	 1.0	(10.99%)
  994	 0.0	 1.0	(10.81%)
  995	 0.0	 1.0	(98.92%)
  996	 0.0	 1.0	(92.23%)
  997	 0.0	 0.0	(49.87%)
  998	 0.0	 1.0	(76.43%)
  999	 0.0	 1.0	(25.3%)


In [77]:
print (float(wrongDetect) / _NUM_RECORDS )*100

82.4


In [ ]:
81.5
resolution 1 w 3 n 20
81.5
resolution 0.88 w 3 n 20

82.4
asli